<a href="https://colab.research.google.com/github/mateokilla/twp/blob/main/gbm_monte_carlo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance tabulate --quiet

import numpy as np
import yfinance as yf
from tabulate import tabulate

# ---- Paraméterek ----
ticker = "VZ"
data = yf.Ticker(ticker).history(period="1y")
S0 = data['Close'].iloc[-1]

# ---- Loghozamok számítása ----
log_returns = np.log(data['Close'] / data['Close'].shift(1)).dropna()

# μ és σ becslése évesítve
mu = log_returns.mean() * 252
sigma = log_returns.std() * np.sqrt(252)

# ---- Monte Carlo szimuláció ----
T = 1         # év
steps = 252
n_sims = 100000
dt = T / steps

np.random.seed(42)
final_prices = []

for sim in range(n_sims):
    price = S0
    for t in range(steps):
        z = np.random.normal()
        price *= np.exp((mu - 0.5*sigma**2)*dt + sigma*np.sqrt(dt)*z)
    final_prices.append(price)

final_prices = np.array(final_prices)

# ---- Percentilisek ----
p5 = np.percentile(final_prices, 5)
p25 = np.percentile(final_prices, 25)
p50 = np.percentile(final_prices, 50)
p75 = np.percentile(final_prices, 75)
p95 = np.percentile(final_prices, 95)

# ---- Szenáriók táblázata ----
scenarios = [
    ["Bear (nagyon rossz, 5%)", f"{p5:.2f} USD", "5%"],
    ["Bearish (rossz, 25%)", f"{p25:.2f} USD", "25%"],
    ["Base (közép, 50%)", f"{p50:.2f} USD", "50%"],
    ["Bullish (jó, 75%)", f"{p75:.2f} USD", "75%"],
    ["Bull (nagyon jó, 95%)", f"{p95:.2f} USD", "95%"]
]

print(f"\n--- {ticker} 1 éves GBM előrejelzés ---")
print(f"Aktuális ár: {S0:.2f} USD")
print(tabulate(scenarios, headers=["Szenárió", "Ár év végén", "Valószínűség"]))
